In [7]:
import kagglehub
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import os
import warnings
warnings.filterwarnings('ignore')

# 딥러닝 관련 라이브러리 import
try:
    import tensorflow as tf
    from tensorflow import keras
    from sklearn.preprocessing import MinMaxScaler, LabelEncoder
    from sklearn.model_selection import train_test_split
    from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
    print("딥러닝 라이브러리 로드 완료")
except ImportError as e:
    print(f"딥러닝 라이브러리 로드 실패: {e}")
    print("설치 명령어: pip install tensorflow scikit-learn")


딥러닝 라이브러리 로드 완료


In [1]:
pip install tensorflow scikit-learn

In [26]:
"""keras.datasets.imdb.load_data(
    path="imdb.npz",
    num_words=None,
    skip_top=0,
    maxlen=None,
    seed=113,
    start_char=1,
    oov_char=2,
    index_from=3,
    **kwargs
) imdb.load_data()함수의 매개변수들

path	"imdb.npz"	데이터 파일명 (내려받거나 저장할 때 사용)
num_words	None	빈도 순위 상위 몇 개 단어만 사용할지. 예: 10000이면 상위 10,000 단어만 사용
skip_top	0	가장 빈도가 높은 단어 중에서 몇 개를 제외할지 (예: 상위 20개 단어 제외)
maxlen	None	각 리뷰 시퀀스 최대 길이 (이 길이를 넘으면 자름)
seed	113	데이터 셔플 시드값 (재현 가능성 위해 설정)
start_char	1	시퀀스 시작을 표시하는 특수 토큰 인덱스
oov_char	2	사전에 없는 단어를 표시하는 특수 토큰 인덱스 (out-of-vocabulary)
index_from	3	실제 단어 인덱스가 몇부터 시작할지 설정 (0,1,2는 특수 토큰에 예약)"""

SyntaxError: unterminated string literal (detected at line 1) (ipython-input-26-881709548.py, line 1)

In [20]:
from tensorflow.keras.datasets import imdb
(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=10000)
#imdb.load_data가 아이템이 2개 있는 2개의 튜플을 반환하는데, 그건 공식문서에 나와있음.
#Tuple of Numpy arrays: (x_train, y_train), (x_test, y_test).
#그걸 받아주는 변수로 지정해주는게 왼쪽의 두가지 튜플 변수.
train_data[0] #0과 1로 구분되어있는데
train_labels[0]

17464789/17464789 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


1

In [21]:
max([max(sequence) for sequence in train_data])

9999

In [23]:
word_index = imdb.get_word_index()#각 단어마다 키와 벨류로 지정이 되어있음
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()]) #"and": 1 이렇게 되어있는걸 반대로 바꿔줌
decoded_review = " ".join(#따로 분리되어있는 아이템들을 띄어쓰기로 보기 좋게 묶어서 한줄로 만들어줌
    [reverse_word_index.get(i - 3, "?") for i in train_data[0]]) #imdb의 word index에서 0,1,2는 특수 토큰이라고 함.
   # 0: 패딩 (padding)
#1: 시작 토큰 (start_char)
#2: OOV(사전에 없는 단어) 토큰

#여기서 패딩은 머신러닝, 딥러닝 처리할때 필요
# 텐서가 기본 단위인데 텐서는 차원의 모양을 띄고 있는 값들을 의미하며 패딩을 준다는 건 길이가 다를 때 길이를 같게 해주는 것임.
#ex)

print(decoded_review)

1641221/1641221 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
? this film was just brilliant casting location scenery story direction everyone's really suited the part they played and you could just imagine being there robert ? is an amazing actor and now the same being director ? father came from the same scottish island as myself so i loved the fact there was a real connection with this film the witty remarks throughout the film were great it was just brilliant so much that i bought the film as soon as it was released for ? and would recommend it to everyone to watch and the fly fishing was amazing really cried at the end it was so sad and you know what they say if you cry at a film it must have been good and this definitely was also ? to the two little boy's that played the ? of norman and paul they were just brilliant children are often left out of the ? list i think because the stars that play them all grown up are such a big profile for the whole film but these children are amazing and should

In [28]:
def vectorize_sequences(sequences, dimension=10000):
    results = np.zeros((len(sequences), dimension))# numpy에서 zeros()함수의 매개변수는
    for i, sequence in enumerate(sequences):
        for j in sequence:
            results[i, j] = 1
    return results
x_train = vectorize_sequences(train_data)
x_test = vectorize_sequences(test_data)
"""매개변수 이름	설명	기본값
shape	생성할 배열의 모양 (튜플이나 정수)	필수
dtype	배열 원소의 데이터 타입	float
order	배열 메모리 저장 순서 ('C' 또는 'F')	'C' (행 우선)
like	호환되는 배열 객체와 비슷한 타입의 배열 생성	None
"""
#인데 위에서 처럼 (n,a)이면 n x a 의 0으로 채워진 행렬데이터셋이 만들어진다.


"매개변수 이름\t설명\t기본값\nshape\t생성할 배열의 모양 (튜플이나 정수)\t필수\ndtype\t배열 원소의 데이터 타입\tfloat\norder\t배열 메모리 저장 순서 ('C' 또는 'F')\t'C' (행 우선)\nlike\t호환되는 배열 객체와 비슷한 타입의 배열 생성\tNone\n"

In [32]:
print(x_train[0])# 0=부정 반응 1 = 긍정반응 이 만개 만들어지는듯
print(len(x_train[0]))

[0. 1. 1. ... 0. 0. 0.]
10000


In [33]:
y_train = np.asarray(train_labels).astype("float32")
y_test=np.asarray(test_labels).astype("float32")

In [34]:
from tensorflow import keras
from tensorflow.keras import layers

model = keras.Sequential([
    layers.Dense(16, activation="relu"),
    layers.Dense(16, activation="relu"),
    layers.Dense(1, activation="sigmoid")

])

In [35]:
model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=["accuracy"])

In [ ]:
x_val = x_train[:10000]
partial_x_train = x_train[]